## References

- http://www.juliaopt.org/SumOfSquares.jl/latest/
- https://www.cs.colorado.edu/~xich8622/papers/thesis.pdf

## Support function of an interval

In this section, as a proof-of-principle, we compute the support function of an interval using SOS.

In [ ]:
using SumOfSquares, DynamicPolynomials, MosekTools, TaylorModels, Plots

The support function of a set $X$ along direction $d$ is defined as the solution of the optimization problem:

$$
\rho(d, X) = \max \langle d, x \rangle \qquad s.t. x \in X.
$$
It represents how much the set $X$ is placed along direction $d$.

In [ ]:
function sf(d::Vector, X::Interval)
    model = SOSModel(with_optimizer(Mosek.Optimizer, QUIET=true))
    @variable(model, x[1])
    @constraint(model, inf(X) <= x[1])
    @constraint(model, x[1] <= sup(X))
    @objective(model, Max, d[1] * x[1])
    optimize!(model)
    
    return objective_value(model)
end

Let's consider an example:

In [ ]:
X = Interval(-2.0, 7.0)

In [ ]:
sf([1.0], X)

In [ ]:
sf([-1.0], X)

## Support function of a TaylorModel1

Now we consider the more general case of a univariate taylor model.

The set defined by a TM is $S := \{ x : x = p(x_0) + y \wedge x_0 \in \textrm{dom(TM)} \wedge y \in \textrm{rem(TM)}\}$.

We would like to compute $\max \langle d, x\rangle$, such that $x \in S$, where $S$ is the range of the taylor model.

To fix ideas, consider the following example.

In [ ]:
p = Taylor1([1.0, 1.0, 1.0], 6)

In [ ]:
rem = Interval(-0.1, 0.1)
x0 = Interval(0, 0)
dom = Interval(-2.0, 2.0)
X = TaylorModel1(p, rem, x0, dom)

In [ ]:
plot(X, lab="")

For the support function along direction $d = [1.0]$, we expect $7.1$, obtained when $x_0 = 2$ and $y = 0.1$.

For the support function along direction $d = [-1.0]$, we expect $0.65$, obtained when $x_0 = -0.5$ and $y = -0.1$.

#### Using Ipopt

In [ ]:
using Ipopt

In [ ]:
d = [1.0]; # direction

In [ ]:
model = Model(with_optimizer(Ipopt.Optimizer, print_level=0))

Xdom = domain(X)
Xrem = remainder(X)
@variable(model, inf(Xdom) <= x0 <= sup(Xdom))
@variable(model, inf(Xrem) <= y <= sup(Xrem))
@variable(model, x)
@NLconstraint(model, x == 1.0 + x0 + 1.0*x0^2 + y)
@objective(model, Max, d[1] * x)
model

In [ ]:
optimize!(model)
@show objective_value(model);

#### Using SumOfSquares

In [ ]:
using SumOfSquares, DynamicPolynomials, MosekTools, SemialgebraicSets

To model the optimization problem using sum-of-squares, we introduce the variable $\gamma$ to represent the objective value bound, implemented as a JuMP decision variable, and introduce the constraint $\langle d, x\rangle \leq \gamma$. The objective function is set to be the minimum over all possible $\gamma$. The minimum upper bound under the polynomial restriction is the value we are looking for.

In [ ]:
model = SOSModel(with_optimizer(Mosek.Optimizer, QUIET=true))
Xdom, Xrem = domain(X), remainder(X)
d = [1.0]

@polyvar x0 x y

S = @set x == 1.0 + x0 + 1.0*x0^2 + y &&
         inf(Xdom) <= x0 && x0 <= sup(Xdom) &&
         inf(Xrem) <= y && y <= sup(Xrem)

@variable(model, γ)
@constraint(model, d[1]*x + 0.0*y + 0.0*x0 <= γ, domain=S, maxdegree=3)

@objective(model, Min, γ)
optimize!(model)
@show objective_value(model);

## Polytopic overapproximation

Here we wrap this approach into a function to compute support functions.

In [2]:
# dependencies
using SumOfSquares, DynamicPolynomials, SemialgebraicSets, MosekTools, SDPA
using Plots
using TaylorModels
using AffineArithmetic

In [4]:
function new_sos(backend, verbose)
    if backend == "Mosek"
        model = SOSModel(with_optimizer(Mosek.Optimizer, QUIET=!verbose))
    elseif backend == "SDPA"
        model = SOSModel(with_optimizer(SDPA.Optimizer)) # TODO QUIET mode?
    else
        error("backend $backend not supported")
    end
    return model
end

# to get runtime:
# MOI.get(model, MOI.SolveTime())
function bounds_SDP(f::Function, dom::Interval, order::Int; backend="Mosek", verbose=false)

    # polynomial variables
    @polyvar x
    p = f(x)

    # box constraints
    B = @set inf(dom) <= x && x <= sup(dom)

    # ============
    # Upper bound
    # ============ 
    model = new_sos(backend, verbose)
    @variable(model, γ) # JuMP decision variable
    @constraint(model, p <= γ, domain=B, maxdegree=order)
    @objective(model, Min, γ)
    optimize!(model)
    upper_bound = objective_value(model)

    # ============
    # Lower bound
    # ============
    model = new_sos(backend, verbose)
    @variable(model, γ) # JuMP decision variable
    @constraint(model, p >= γ, domain=B, maxdegree=order)
    @objective(model, Max, γ)
    optimize!(model)
    lower_bound = objective_value(model)

    return Interval(lower_bound, upper_bound)

end

function bounds_SDP(f::Function, dom::IntervalBox{N}, order::Int; backend="Mosek", verbose=false) where {N}
    
    # polynomial variables
    @polyvar x[1:N]
    p = f(x...)

    # box constraints
    Bi =[@set inf(dom[i]) <= x[i] && x[i] <= sup(dom[i]) for i in 1:N]
    B = reduce(intersect, Bi)

    # ============
    # Upper bound
    # ============ 
    model = new_sos(backend, verbose)
    @variable(model, γ) # JuMP decision variable
    @constraint(model, p <= γ, domain=B, maxdegree=order)
    @objective(model, Min, γ)
    optimize!(model)
    upper_bound = objective_value(model)

    # ============
    # Lower bound
    # ============
    model = new_sos(backend, verbose)
    @variable(model, γ) # JuMP decision variable
    @constraint(model, p >= γ, domain=B, maxdegree=order)
    @objective(model, Max, γ)
    optimize!(model)
    lower_bound = objective_value(model)

    return Interval(lower_bound, upper_bound)
end

bounds_SDP (generic function with 2 methods)

In [5]:
func(x) = x^2 - 1
bounds_SDP(func, Interval(-2.0, 1.0), 3)

[-1.00001, 3]

In [6]:
func(x, y) = x^2*y^2 - 1
bounds_SDP(func, IntervalBox(-2.0..1.0, 3.0..5.0), 4)

[-0.999999, 99]

In [13]:
poly(x) = -x*x*x/6.0
dom = Interval(-4.5, -0.3)
bounds_SDP(func, dom, 5)

[0.00450004, 15.1875]

[-2.4, -2.39999]

In [41]:
poly(x0, y) = -x0*x0*x0/6.0 + y

rem = Interval(0.0, 0.0)
dom = Interval(-4.5, -0.3) × rem
d = [1.0]
bnd = sup(bounds_SDP((x0, y) -> poly(d[1]*x0, d[1]*y), dom, 5))

15.187499893161952

search: bounds_SDP



No documentation found.

`bounds_SDP` is a `Function`.

```
# 2 methods for generic function "bounds_SDP":
[1] bounds_SDP(f::Function, dom::Interval, order::Int64; backend, verbose) in Main at In[4]:17
[2] bounds_SDP(f::Function, dom::IntervalBox{N,T} where T, order::Int64; backend, verbose) where N in Main at In[4]:50
```


Now we consider the more general case of a univariate taylor model.

The set defined by a TM is $S := \{ x : x = p(x_0) + y \wedge x_0 \in \textrm{dom(TM)} \wedge y \in \textrm{rem(TM)}\}$.

We would like to compute $\max \langle d, x\rangle$, such that $x \in S$, where $S$ is the range of the taylor model.

Consider the following example: the taylor model `(p, I)` where `p(x) = -x^3/6` defined over `dom = [-4.5, -0.3]`. Let the interval remainder be zero, `I = Interval(0.0, 0.0)`.

In [9]:
dom = Interval(-4.5, -0.3)
func(x) = -x*x*x/6.0
ord = 5
x0 = Interval(mid(dom))
x = TaylorModel1(ord, x0, dom)
tm = func(x) # evaluate(func(x), dom - x0)
tmpol = tm.pol

 [2.30399, 2.30401] + [-2.88001, -2.87999] t + [1.19999, 1.20001] t² + [-0.166667, -0.166666] t³ + 𝒪(t⁶)

In [10]:
d = [1.0]
order = 4
@polyvar x0pol y

# for the conversion from interval coeffs to floating points we take the
# midpoint of the interval
p = sum(x0pol^(i-1) * inf(tmpol.coeffs[i]) for i in 1:get_order(tmpol)+1)

B = @set inf(dom) <= x0pol && x0pol <= sup(dom) &&
         y <= sup(tm.rem) && y >= inf(tm.rem)

model = new_sos("Mosek", false)
@variable(model, γ) # JuMP decision variable <d, x> <= γ
@constraint(model, d[1] * p + d[1] * y <= γ, domain=B, maxdegree=order)
@objective(model, Min, γ)

model

A JuMP Model
Minimization problem with:
Variable: 1
Objective function type: VariableRef
`Array{GenericAffExpr{Float64,VariableRef},1}`-in-`SumOfSquares.SOSPolynomialSet{BasicSemialgebraicSet{Float64,Polynomial{true,Float64},AlgebraicSet{Float64,Polynomial{true,Float64},Buchberger,SemialgebraicSets.SolverUsingMultiplicationMatrices{SemialgebraicSets.GröbnerBasisMultiplicationMatricesAlgorithm,ReorderedSchurMultiplicationMatricesSolver{Float64,Random.MersenneTwister}}}},NonnegPolyInnerCone{MathOptInterface.PositiveSemidefiniteConeTriangle},MonomialBasis,Monomial{true},MonomialVector{true},Tuple{}}`: 1 constraint
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Mosek
Names registered in the model: γ

In [11]:
optimize!(model)

objective_value(model)

54.75149238366741

In [ ]:
# another test: try bounds_SDP function
sf(x) = x -> d[1] * x

In [ ]:
model = SOSModel(with_optimizer(Mosek.Optimizer, QUIET=true))
Xdom, Xrem = domain(X), remainder(X)
d = [1.0]

@polyvar x0 x y

S = @set x == 1.0 + x0 + 1.0*x0^2 + y &&
         inf(Xdom) <= x0 && x0 <= sup(Xdom) &&
         inf(Xrem) <= y && y <= sup(Xrem)

@variable(model, γ)
@constraint(model, d[1]*x + 0.0*y + 0.0*x0 <= γ, domain=S, maxdegree=3)

@objective(model, Min, γ)
optimize!(model)
@show objective_value(model);

In [ ]:
function bounds_SDP(f::Function, dom::Interval, order::Int; backend="Mosek", verbose=false)

    # polynomial variables
    @polyvar x
    p = f(x)

    # box constraints
    B = @set inf(dom) <= x && x <= sup(dom)

    # ============
    # Upper bound
    # ============ 
    model = new_sos(backend, verbose)
    @variable(model, γ) # JuMP decision variable
    @constraint(model, p <= γ, domain=B, maxdegree=order)
    @objective(model, Min, γ)
    optimize!(model)
    upper_bound = objective_value(model)

    # ============
    # Lower bound
    # ============
    model = new_sos(backend, verbose)
    @variable(model, γ) # JuMP decision variable
    @constraint(model, p >= γ, domain=B, maxdegree=order)
    @objective(model, Max, γ)
    optimize!(model)
    lower_bound = objective_value(model)

    return Interval(lower_bound, upper_bound)

end

In [ ]:
function sf(d::Vector, X::Vector{Taylor1})
    model = Model(with_optimizer(Ipopt.Optimizer, print_level=0))

    Xdom = domain(X)
    Xrem = remainder(X)
    @variable(model, inf(Xdom) <= x0 <= sup(Xdom))
    @variable(model, inf(Xrem) <= y <= sup(Xrem))
    @variable(model, x)
    @NLconstraint(model, x == sum(x0^i * p.coeffs[i] for i in 1:1+get_order(p)) + y)
    @objective(model, Max, d[1] * x)
    model
end

In [ ]:
@NLconstraint(model, sum(x0^i * p.coeffs[i] for i in 1:1+get_order(p)) == 0)

In [ ]:
S = sum(x0^i * p.coeffs[i] for i in 1:1+get_order(p))